<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/twadr_6_mixed_pubmed_bigtweet_twadr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-6.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-6.validation.csv",header=None)
valid.head()

,0,1
0,13,It's 6 am and this adderall got me reevaluatin...
1,84,paraplegic
2,91,jitters
3,189,dizziness
4,189,dizzy


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-6.test.csv",header=None)
test.head()

,0,1
0,13,awake in the middle of the night for an hour or 2
1,24,heart beat for being too strong
2,74,60% of my success is due to taking Adderall
3,85,tremor
4,94,anxiety


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_twadr.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'for',
 'with',
 'was',
 'diclofenac',
 'were',
 'on',
 'is',
 'rt',
 'pain',
 'sodium',
 'by',
 'drug',
 'that',
 'as',
 'i',
 'at',
 'patients',
 'this',
 'or',
 'from',
 'you',
 'disease',
 'be',
 'increased',
 'mg',
 'group',
 'my',
 'study',
 'mood',
 'are',
 'not',
 'treatment',
 'it',
 'an',
 'me',
 'after',
 'amp',
 'have',
 'p',
 'release',
 'gluten',
 ',',
 'disorder',
 'all',
 'abnormal',
 'effect',
 'withdrawal',
 'we',
 'than',
 'symptoms',
 'free',
 'no',
 'decreased',
 'weight',
 'new',
 'using',
 'compared',
 'out',
 'more',
 'anxiety',
 'your',
 'day',
 'effects',
 'time',
 'groups',
 'our',
 'two',
 'up',
 'both',
 'about',
 'inflammatory',
 'blood',
 'significantly',
 'drugs',
 'like',
 'between',
 'h',
 'its',
 'these',
 'potassium',
 'acute',
 'had',
 'significant',
 'but',
 'skin',
 'anti',
 'use',
 'disorders',
 'which',
 'can',
 '

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.732399,4.188763,0.346638,23:13


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.012892,3.810110,0.383381,23:09


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.728695,3.765530,0.387649,22:57


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.678475,3.732365,0.391058,23:03


In [25]:
learn.save_encoder('pubmed_cadec_twadrl_mixed_fold6_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_cadec_twadrl_mixed_fold6_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.137756,6.667123,0.089286,00:22


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.198231,5.462711,0.107143,00:27


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.734224,5.149592,0.125000,01:10


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.436070,4.900697,0.142857,01:05
1,5.209563,4.773376,0.151786,01:08


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.922633,4.561675,0.178571,01:09
1,4.752009,4.491812,0.187500,01:07


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.518202,4.375848,0.196429,01:04
1,4.375280,4.337149,0.187500,01:09


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.176062,4.259490,0.214286,01:04
1,4.025954,4.192548,0.205357,01:06


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.846878,4.152839,0.214286,01:06
1,3.695011,4.031169,0.258929,01:07


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.497334,3.998306,0.258929,01:11
1,3.424232,3.985401,0.303571,01:04
2,3.252732,3.828943,0.357143,01:09
3,3.066812,3.846464,0.366071,01:05


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.864336,3.756300,0.348214,01:06
1,2.813767,3.778448,0.401786,01:05
2,2.637537,3.742573,0.392857,01:07
3,2.492473,3.777234,0.392857,01:07


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.390920,3.749966,0.375000,01:07
1,2.313315,3.759558,0.383929,01:10
2,2.153415,3.747515,0.392857,01:04
3,1.996921,3.730065,0.383929,01:06


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.870489,3.753377,0.392857,01:07
1,1.834096,3.984531,0.383929,01:07
2,1.697580,4.040472,0.410714,01:04
3,1.580660,3.980116,0.383929,01:05


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.496447,3.947343,0.383929,01:08
1,1.474517,3.976120,0.383929,01:07
2,1.336496,4.145883,0.366071,01:04
3,1.226804,4.040338,0.392857,01:08


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.124418,4.119080,0.383929,01:07
1,1.105890,4.318827,0.366071,01:04
2,1.023191,4.259465,0.401786,01:08
3,0.949756,4.333804,0.401786,01:06


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.899530,4.350904,0.401786,01:10
1,0.908967,4.661819,0.348214,01:11
2,0.844703,4.580027,0.348214,01:05
3,0.756089,4.573040,0.375000,01:06


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.718689,4.569120,0.383929,01:05
1,0.699477,4.567524,0.401786,01:05


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.668746,4.632843,0.375000,01:10
1,0.648184,4.588383,0.383929,01:09


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.644740,4.766668,0.392857,01:05


In [44]:
# save the best model

learn.save_encoder('pubmed_cadec_twadrl_mixed_fold6')

In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.632201,4.804320,0.383929,01:06


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.608393,4.715539,0.383929,01:08


In [47]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.551251,4.944129,0.401786,01:07


# Part three: Predict on the test dataset

In [48]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[2032, 1718, 2097, 1204, 978, 1096, 189, 1594, 189, 217, 224, 2103, 1578, 1936, 1936, 2147, 1768, 1327, 1599, 1136, 426, 532, 1787, 1371, 1371, 706, 1787, 1711, 755, 504, 792, 809, 1264, 2032, 1787, 824, 1099, 648, 908, 1278, 923, 1940, 974, 1325, 978, 978, 1822, 978, 2147, 1718, 1669, 187, 1073, 1099, 1769, 1214, 1787, 945, 1529, 1594, 1509, 561, 1204, 2023, 250, 1265, 1265, 1822, 1295, 2147, 1308, 1164, 371, 1340, 1925, 1458, 1787, 2194, 1327, 1711, 2173, 651, 1597, 1597, 1546, 1597, 1610, 1610, 1822, 1680, 1787, 1727, 2191, 1787, 265, 1727, 1727, 224, 1767, 1718, 1669, 1727, 1427, 1787, 1787, 1787, 1787, 1787, 1787, 978, 56, 824, 1884, 426, 1936, 1936, 233, 1936, 1857, 452, 1925, 452, 1798, 452, 2018, 2024, 1787, 2147, 2097, 1508, 1508, 2149, 224, 2147, 2147, 974, 233, 2147, 2147, 2147, 2149, 1521, 1521]
[13, 24, 74, 85, 94, 187, 189, 189, 189, 217, 224, 224, 224, 233, 233, 233, 265, 355, 392, 396, 426, 532, 561, 600, 600, 706, 744, 749, 755, 791, 792, 809, 818, 818, 819, 82

In [50]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

51
0.35664335664335667
